In [76]:
import pandas as pd

In [185]:
def _combine_data(path='data/'):
    # Read raw data
    train = pd.read_csv(path+'contraceptive_logistics_data.csv')
    location = pd.read_csv(path+'service_delivery_site_data.csv')
    product = pd.read_csv(path+'product.csv')

    # Expand data frame
    month_year = train[['year','month']].drop_duplicates().reset_index(drop=True)
    product_site = train[['region','district','site_code','product_code']].drop_duplicates().reset_index(drop=True)
    train_base = pd.merge(month_year.assign(j=1), product_site.assign(j=1)).drop(columns = 'j')
    train = pd.merge(train_base, train, how='left')

    # Add date and index
    train['day'] = 1
    train['ds'] = pd.to_datetime(train[['year','month','day']])
    train = train.sort_values(by=['site_code','product_code','ds']).reset_index(drop=True)
    train['isna'] = train['stock_distributed'].isna()
    train['idx'] = train.groupby(['site_code','product_code'])['ds'].rank(method='first', ascending=True)
    train = train.drop(columns = ['year','month','day'])

    # Join with location
    train = pd.merge(train, location.drop(columns=['site_region','site_district']))

    # Join with product
    train = pd.merge(train, product)

    # Rearrange columns
    train = train[['site_code','product_code'] + train.drop(columns=['site_code','product_code']).columns.tolist()]

    # Change category
    train = train.sort_values(by=['site_code','product_code','ds']).reset_index(drop=True)
    train['idx'] = train['idx'].astype(int)
    train['ds'] = train['ds'].dt.date.astype(str)
    train['product_name'] = train['product_name'].str.strip()
    
    return(train)

In [186]:
df2 = _combine_data()

In [187]:
df = pd.read_csv('data/ifc_clean.csv')

In [188]:
for col in df.columns.tolist():
    print(col, sum(df[col] == df2[col]))

site_code 61065
product_code 61065
region 61065
district 61065
stock_initial 38842
stock_received 38842
stock_distributed 38842
stock_adjustment 38842
stock_end 38842
average_monthly_consumption 38842
stock_stockout_days 38842
stock_ordered 38072
ds 61065
isna 61065
idx 61065
site_type 61065
site_latitude 61065
site_longitude 61065
product_type 61065
product_name 61065
